In [1]:
import json
import numpy as np
import pandas as pd
import os

import importlib
from surrogate_rule import forest_info
from surrogate_rule import tree_node_info
importlib.reload(tree_node_info)
importlib.reload(forest_info)

import altair as alt


In [30]:
data_list = [
    "penn_wind", 
    "penn_cpu", 
    "penn_satellite", 
    "crime", 
    "diabetes",
    "loan",
    "income",
    "dry_bean",
    "obesity_level",
    "music_origin"
]



In [3]:
# define pruning constraints

filter_threshold = {
    "support": 5,
    "fidelity": .85,
    "num_feat": 4,
    "num_bin": 3,
}
num_bin = filter_threshold['num_bin']


## SuRE

In [4]:
import importlib
from surrogate_rule import forest_info
from surrogate_rule import tree_node_info
importlib.reload(tree_node_info)
importlib.reload(forest_info)

<module 'surrogate_rule.forest_info' from '/Users/junyuan/Documents/_/python/sure_eval/surrogate_rule/forest_info.py'>

In [27]:
def extract_rules_from_RF():
    # train surrogate
    surrogate_obj = tree_node_info.tree_node_info()

    surrogate_obj.initialize(X=np.array(data['data']), y=np.array(data['y_gt']), 
                             y_pred=np.array(data['y_pred']), debug_class=-1,
                             attrs=data['columns'], filter_threshold=filter_threshold,
                             n_cls=len(data['target_names']),
                             num_bin=num_bin, verbose=False
    ).train_surrogate_random_forest().tree_pruning()

    forest_obj = tree_node_info.forest()
    forest_obj.initialize(
        trees=surrogate_obj.tree_list, cate_X=surrogate_obj.cate_X, 
        y=surrogate_obj.y, y_pred=surrogate_obj.y_pred, attrs=attrs, num_bin=num_bin,
        real_percentiles=surrogate_obj.real_percentiles,
        real_min=surrogate_obj.real_min, real_max=surrogate_obj.real_max,
    ).construct_tree().extract_rules()
    
    forest = forest_info.Forest()

    forest.initialize(forest_obj.tree_node_dict, data['real_min'], data['real_max'], surrogate_obj.percentile_info,
        df, data['y_pred'], data['y_gt'],
        forest_obj.rule_lists,
        data['target_names'], 2)
    forest.initialize_rule_match_table()
    forest.initilized_rule_overlapping()
    try:
        res = forest.find_the_min_set()
    except:
        return False
    lattice = forest.get_lattice_structure(res['rules'])
    
    max_feat = 0
    min_feat = 111
    avg_feat = 0.0
    for rule in res['rules']:
        if (len(rule['rules']) > max_feat):
            max_feat = len(rule['rules'])
        if (len(rule['rules']) < min_feat):
            min_feat = len(rule['rules'])
        avg_feat += len(rule['rules'])
                           
    return [len(res['rules']), res['coverage'], max_feat, min_feat, avg_feat/len(res['rules']), len(lattice)]

## Decision Tree

In [28]:
# train surrogate DT
def extract_rules_from_DT():
    surrogate_obj2 = tree_node_info.tree_node_info()

    surrogate_obj2.initialize(X=np.array(data['data']), y=np.array(data['y_gt']), 
                             y_pred=np.array(data['y_pred']), debug_class=-1,
                             attrs=data['columns'], filter_threshold=filter_threshold,
                             n_cls=len(data['target_names']),
                             num_bin=num_bin, verbose=False
    ).train_surrogate_decision_tree().tree_pruning(False)
    
    tree_obj = tree_node_info.forest()

    tree_obj.initialize(
        trees=surrogate_obj2.tree_list, cate_X=surrogate_obj2.cate_X, 
        y=surrogate_obj2.y, y_pred=surrogate_obj2.y_pred, attrs=attrs, num_bin=num_bin,
        real_percentiles=surrogate_obj2.real_percentiles,
        real_min=surrogate_obj2.real_min, real_max=surrogate_obj2.real_max,
    ).construct_tree().extract_rules()

    dt = forest_info.Forest()

    dt.initialize(tree_obj.tree_node_dict, data['real_min'], data['real_max'], surrogate_obj2.percentile_info,
        df, data['y_pred'], data['y_gt'],
        tree_obj.rule_lists,
        data['target_names'], 2)
    dt.initialize_rule_match_table()
    dt.initilized_rule_overlapping()
    try:
        res = dt.find_the_min_set()
    except:
        return False
    max_feat = 0
    min_feat = 200
    avg_feat = 0.0
    for rule in res['rules']:
        if (len(rule['rules']) > max_feat):
            max_feat = len(rule['rules'])
        if (len(rule['rules']) < min_feat):
            min_feat = len(rule['rules'])
        avg_feat += len(rule['rules'])
                           
    return [len(res['rules']), res['coverage'], max_feat, min_feat, avg_feat/len(res['rules']), len(tree_obj.tree_node_dict)]

In [8]:
num_feat_list = [1,2,3,4, 5, 6,7,8,9,10]
num_bin = filter_threshold['num_bin']


for data_name in data_list:
    file = "output/" + data_name + "/test.json"
    with open(file, 'r') as json_input:
        data = json.load(json_input)
    attrs = data['columns']
    

In [9]:
len(data['data'])

1688

In [10]:
data.keys()

dict_keys(['columns', 'data', 'target_names', 'real_min', 'real_max', 'y_pred', 'y_gt', 'n_cls'])

In [40]:
# read json
to_plot_list = []
filter_threshold = {
    "support": 5,
    "fidelity": .85,
    "num_feat": 4,
    "num_bin": 3,
}

# num_feat_list = [2,3, 4, 5, 6,7]
num_feat_list = [1,2,3,4, 5, 6,7,8,9,10]
num_bin = filter_threshold['num_bin']


for data_name in data_list:
    file = "output/" + data_name + "/test.json"
    with open(file, 'r') as json_input:
        data = json.load(json_input)
    attrs = data['columns']
    df = pd.DataFrame(data=np.array(data['data']), columns = attrs)
    y_pred = data['y_pred']
    res = []
    for num_feat in num_feat_list:
        print("========", num_feat, data_name, "========")
        filter_threshold['num_feat'] = num_feat
        
        res_sure = extract_rules_from_RF()
        res_dt = extract_rules_from_DT()
        
        if (not res_sure):
            print("sure error")
        else:
            sure_len, sure_cover, sure_max_feat, sure_min_feat, sure_avg_feat, lattice_size = res_sure
            res.append({
                'method': 'HSR',
                'len': sure_len,
                'cover': sure_cover,
                'max_feat': sure_max_feat,
                'min_feat': sure_min_feat,
                'avg_feat': sure_avg_feat,
                'num_feat': num_feat,
                'lattice/tree_size': lattice_size,
            })
            
        if (not res_dt):
            print("dt error")
        else:
            dt_len, dt_cover, dt_max_feat, dt_min_feat, dt_avg_feat, tree_size = res_dt
            res.append( {
                'method': 'SDT',
                'len': dt_len,
                'cover': dt_cover,
                'max_feat': dt_max_feat,
                'min_feat': dt_min_feat,
                'avg_feat': dt_avg_feat,
                'num_feat': num_feat,
                'lattice/tree_size': tree_size,
            })
            
    with open("exp_output/compare_"+data_name+".json", "w") as output:
        output.write(json.dumps({"res": res}))

======== 1 penn_wind ========
======== 2 penn_wind ========
======== 3 penn_wind ========
======== 4 penn_wind ========
======== 5 penn_wind ========
======== 6 penn_wind ========
======== 7 penn_wind ========
======== 8 penn_wind ========
======== 9 penn_wind ========
======== 10 penn_wind ========
======== 1 penn_cpu ========
======== 2 penn_cpu ========
======== 3 penn_cpu ========
======== 4 penn_cpu ========
======== 5 penn_cpu ========
======== 6 penn_cpu ========
======== 7 penn_cpu ========
======== 8 penn_cpu ========
======== 9 penn_cpu ========
======== 10 penn_cpu ========
======== 1 penn_satellite ========
======== 2 penn_satellite ========
======== 3 penn_satellite ========
======== 4 penn_satellite ========
======== 5 penn_satellite ========
======== 6 penn_satellite ========
======== 7 penn_satellite ========
======== 8 penn_satellite ========
======== 9 penn_satellite ========
======== 10 penn_satellite ========
======== 1 crime ========
======== 2 crime ========
=====

# Plotting

In [41]:
to_plot_list = []

data_name_list = [
 'crime',
 'diabetes',
 'dry_bean',
 'income',
 'loan',
 'obesity_level',
 'penn_cpu',
 'penn_satellite',
 'penn_wind',
 "music_origin"
]


for data_name in data_name_list:
    with open("exp_output/compare_" + data_name +'.json', 'r') as json_input:
        data = json.load(json_input)
        result = data['res']
    for r in result:
        to_plot_list.append([data_name, r['method'], r['len'], r['cover'], r['max_feat'], r['min_feat'], r['avg_feat'], r['num_feat'], r['lattice/tree_size']])
#         to_plot_list.append([data_name, 'DT', r['len'], r['cover'], r['max_feat'], r['min_feat'], r['avg_feat'], r['num_feat'], r['lattice/tree_size']])
        


In [42]:
data_name_list.sort()
data_name_list

['crime',
 'diabetes',
 'dry_bean',
 'income',
 'loan',
 'music_origin',
 'obesity_level',
 'penn_cpu',
 'penn_satellite',
 'penn_wind']

In [43]:
to_plot = pd.DataFrame(data=to_plot_list, columns=['data_name', 'method', 'set_size', 'coverage', 'max_feat', 'min_feat', 'avg_feat', 'n_feat', 'node_num'])

In [44]:
to_plot

,data_name,method,set_size,coverage,max_feat,min_feat,avg_feat,n_feat,node_num
0,crime,HSR,12,0.979310,1,1,1.000000,1,13
1,crime,SDT,1,0.333542,1,1,1.000000,1,3
2,crime,HSR,11,1.000000,2,1,1.454545,2,16
3,crime,SDT,2,0.668966,2,1,1.500000,2,5
4,crime,HSR,9,0.997492,3,1,2.000000,3,18
...,...,...,...,...,...,...,...,...,...
191,music_origin,SDT,16,0.932704,8,1,5.875000,8,42
192,music_origin,HSR,18,0.989374,9,1,4.222222,9,73
193,music_origin,SDT,17,0.938607,9,1,6.117647,9,44
194,music_origin,HSR,19,0.994097,10,1,4.368421,10,79


In [45]:
to_plot['coverage'] = to_plot['coverage'].round(2)
to_plot['avg_feat'] = to_plot['avg_feat'].round(2)

In [46]:
to_plot = to_plot.rename(columns={"set_size": "number of rules",
                       "avg_feat": "average conditions"})

In [47]:
to_plot

,data_name,method,number of rules,coverage,max_feat,min_feat,average conditions,n_feat,node_num
0,crime,HSR,12,0.98,1,1,1.00,1,13
1,crime,SDT,1,0.33,1,1,1.00,1,3
2,crime,HSR,11,1.00,2,1,1.45,2,16
3,crime,SDT,2,0.67,2,1,1.50,2,5
4,crime,HSR,9,1.00,3,1,2.00,3,18
...,...,...,...,...,...,...,...,...,...
191,music_origin,SDT,16,0.93,8,1,5.88,8,42
192,music_origin,HSR,18,0.99,9,1,4.22,9,73
193,music_origin,SDT,17,0.94,9,1,6.12,9,44
194,music_origin,HSR,19,0.99,10,1,4.37,10,79


In [48]:
alt.Chart(to_plot).mark_line().encode(
    x=alt.X('n_feat:Q', axis=alt.Axis(tickCount=6,)),
    y=alt.Y(alt.repeat('row'), type='quantitative', axis=alt.Axis(tickCount=5,)),
    color="method:N",
).properties(
    width=80,
    height = 80,
).facet(
    column=alt.Column('data_name:N', title=None),
).repeat(
    row=['coverage', 'number of rules', 'average conditions'],
)

alt.RepeatChart(...)

In [49]:
alt.Chart(to_plot).mark_line().encode(
    x=alt.X('n_feat:Q', axis=alt.Axis(tickCount=6,title=None)),
    y=alt.Y(alt.repeat('row'), type='quantitative', axis=alt.Axis(tickCount=5,title=None)),
    color="method:N",
).properties(
    width=80,
    height = 80,
).facet(
    column=alt.Column('data_name:N', title=None),
).repeat(
#     row=['coverage', 'avg_feat', 'set_size', 'node_num']
    row=['coverage', 'number of rules', 'average conditions'],
)

alt.RepeatChart(...)

In [216]:
for data_name in data_name_list:
    file = "output/" + data_name + "/test.json"
    with open(file, 'r') as json_input:
        data = json.load(json_input)
    print(data_name)
    print("shape=", np.array(data['data']).shape, "#class=", len(data['target_names']))

crime
shape= (1595, 101) #class= 2
diabetes
shape= (1000, 8) #class= 2
dry_bean
shape= (10888, 16) #class= 7
income
shape= (12545, 6) #class= 2
loan
shape= (6317, 22) #class= 2
music_origin
shape= (847, 117) #class= 2
obesity_level
shape= (1688, 16) #class= 7
penn_cpu
shape= (6553, 21) #class= 2
penn_satellite
shape= (5148, 36) #class= 2
penn_wind
shape= (5259, 14) #class= 2
